In [1]:
import hypernetx as hnx # make sure you're in the hyperbio branch of hnx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# make sure you modify these file names and locations as appropriate
datadir = 'C:/Users/hoga886/Documents/_Projects_code_work/HyperBio/Data/Mitchell/'
datafile = datadir+'bigTrans_eap.txt'
metafile = datadir+'transMeta.txt'
labelfile = datadir+'bigTransLabels.txt'

## Read the data file into a pandas array 
df = pd.read_csv(datafile, sep='\t')
df = df.set_index('Protein')

## Read the metadata file into a pandas array
df_meta = pd.read_csv(metafile, sep='\t')
df_meta = df_meta.set_index(['column_names'])

## Read the labels file (similar to metadata) into a pandas array
df_labels = pd.read_csv(labelfile, sep='\t')
df_labels = df_labels.set_index(['samples'])

# find the sets of columns corresponding to the b values
# and the p values separately, and make a translation dictionary
df_b_cols = []
df_p_cols = []
bp_dict = {}
for row in df_meta.itertuples():
    if (('__b' in row.Index) or ('Mock_b' in row.Index) or ('._b' in row.Index)) and row.Index in df.columns:
        df_b_cols.append(row.Index)
        
        if '__b' in row.Index:
            p_name = row.Index.replace('__b', '__p')
        elif 'Mock_b' in row.Index:
            p_name = row.Index.replace('Mock_b', 'Mock_p')
        elif '._b' in row.Index:
            p_name = row.Index.replace('._b', '._p')
        bp_dict[row.Index] = p_name
    elif (('__p' in row.Index) or ('Mock_p' in row.Index) or ('._p' in row.Index)) and row.Index in df.columns:
        df_p_cols.append(row.Index)
    else:
        print(row.Index)

In [3]:
df.head()

,EB1_WT_0h__b,EB1_WT_0h__p,EB1_WT_00h__b,EB1_WT_00h__p,EB1_WT_8h__b,EB1_WT_8h__p,EB1_WT_18h__b,EB1_WT_18h__p,EB1_WT_24h__b,EB1_WT_24h__p,...,mouse_ln_WNV_WT_6d__b,mouse_ln_WNV_WT_6d__p,mouse_ln_WNV_E218A_1d__b,mouse_ln_WNV_E218A_1d__p,mouse_ln_WNV_E218A_2d__b,mouse_ln_WNV_E218A_2d__p,mouse_ln_WNV_E218A_4d__b,mouse_ln_WNV_E218A_4d__p,mouse_ln_WNV_E218A_6d__b,mouse_ln_WNV_E218A_6d__p
Protein,,,,,,,,,,,,,,,,,,,,,
AAAS,-0.053539,0.897905,-0.021629,0.971758,0.069450,0.893481,0.390366,0.150667,0.073836,0.705193,...,0.818601,0.199452,0.675964,0.000681,1.295524,0.000307,1.474642,0.000588,1.359757,0.012836
AACS,0.031504,0.896717,0.131252,0.409202,0.309998,0.244921,0.401375,0.037433,0.165346,0.354110,...,-1.549177,0.038906,-0.788553,0.123942,-0.743268,0.038090,-0.632166,0.112165,-1.220868,0.041723
AADAC,-0.041660,0.837731,0.031732,0.905520,-0.106712,0.351493,-0.224353,0.169825,-0.045614,0.834415,...,0.036319,0.768323,0.030294,0.781156,0.030260,0.748385,0.020523,0.891356,0.009379,0.963882
AAK1,0.139425,0.507731,0.148185,0.805341,0.251240,0.196918,0.290396,0.058642,0.383917,0.082190,...,0.235774,0.359214,0.871901,0.009966,1.341327,0.034060,-0.485618,0.168506,-1.095028,0.100790
AAMP,0.139837,0.470499,0.005684,0.988458,0.077773,0.496487,0.128518,0.292702,0.132485,0.179721,...,-1.181211,0.237053,0.389658,0.346917,-0.150656,0.560934,-0.244075,0.553876,-0.914955,0.221456


In [4]:
# What are all of the hosts?
df_meta['host'].unique()

array(['Huh cells', 'Calu3 cells', 'mouse lung', 'Human fibroblasts',
       'Human airway epithelium', 'Human microvascular endothelium',
       'mouse cortex', 'mouse cerebellum', 'mouse lymph node'],
      dtype=object)

In [5]:
# find the b and p columns corresponding to human data
human_hosts = ['Huh cells', 'Calu3 cells', 'Human fibroblasts',
       'Human airway epithelium', 'Human microvascular endothelium']
human_cols = df_meta[df_meta['host'].isin(human_hosts)].index
human_b_cols = list(human_cols.intersection(df_b_cols))
human_p_cols = list(human_cols.intersection(df_p_cols))
len(human_b_cols), len(human_p_cols)

(184, 184)

## Creating the hypergraph
For bigTrans on its own we have both fold change (b) columns and p-value columns. We were creating a hypergraph using a hybrid filter: z-score > 2 and p-value <= 0.05. This requires some pandas manipulation to get to a dataframe of True/False values that can be turned into a hypergraph. For biggerTrans we won't have the p-values so we won't use the hybrid filter. You can still do the datarame transformation outside of the from_dataframe command, or you can give a list of arguments to from_dataframe and have that function do the work.

### Option 1: b-value and p-value filtering (just so you can see how I did it)

In [6]:
from scipy.stats import zscore

# restrict the data to the human b columns
df_human = df[human_b_cols]

# apply the absolute value of z score to the columns
df_human = abs(df_human.apply(zscore))

# apply the z-score > 2 filter
df_human = df_human.apply(lambda x : x > 2)

# restrict the data to the p columns
df_human_p = df[human_p_cols]

# apply the p value <= 0.05 filter
df_human_p = df_human_p.apply(lambda x : x <= 0.05)

# create a final dataframe combining the z-score and p-value filters
df_final = pd.DataFrame()
for col in df_human:
    df_final[col] = df_human[col] & df_human_p[bp_dict[col]]
    
df_final.head()

,EB1_WT_0h__b,EB1_WT_00h__b,EB1_WT_8h__b,EB1_WT_18h__b,EB1_WT_24h__b,EB1_WT_48h__b,EB1_mucin_0h__b,EB1_mucin_00h__b,EB1_mucin_8h__b,EB1_mucin_18h__b,...,calu3_SARS__b_ExoNI_moi1_60h,calu3_SARS__b_ExoNI_moi1_72h,calu3_SARS__b_nsp16_moi5_0h,calu3_SARS__b_nsp16_moi5_7h,calu3_SARS__b_nsp16_moi5_12h,calu3_SARS__b_nsp16_moi5_24h,calu3_SARS__b_nsp16_moi5_36h,calu3_SARS__b_nsp16_moi5_48h,calu3_SARS__b_nsp16_moi5_60h,calu3_SARS__b_nsp16_moi5_72h
Protein,,,,,,,,,,,,,,,,,,,,,
AAAS,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
AACS,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AADAC,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AAK1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
AAMP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
df_final.shape

(9760, 184)

In [8]:
# create the hypergraph using from_dataframe with no additional arguments
H_bp_filter = hnx.Hypergraph.from_dataframe(df_final)

In [9]:
H_bp_filter.shape

(6882, 150)

### Option 2: b-value filtering by hand, so you can see the steps

In [10]:
# restrict the data to the human b columns
df_human2 = df[human_b_cols]

# apply the absolute value of z score to the columns
df_human2 = abs(df_human2.apply(zscore))

# apply the z-score > 2 filter
df_human2 = df_human2.apply(lambda x : x > 2)

df_human2.head()

,EB1_WT_0h__b,EB1_WT_00h__b,EB1_WT_8h__b,EB1_WT_18h__b,EB1_WT_24h__b,EB1_WT_48h__b,EB1_mucin_0h__b,EB1_mucin_00h__b,EB1_mucin_8h__b,EB1_mucin_18h__b,...,calu3_SARS__b_ExoNI_moi1_60h,calu3_SARS__b_ExoNI_moi1_72h,calu3_SARS__b_nsp16_moi5_0h,calu3_SARS__b_nsp16_moi5_7h,calu3_SARS__b_nsp16_moi5_12h,calu3_SARS__b_nsp16_moi5_24h,calu3_SARS__b_nsp16_moi5_36h,calu3_SARS__b_nsp16_moi5_48h,calu3_SARS__b_nsp16_moi5_60h,calu3_SARS__b_nsp16_moi5_72h
Protein,,,,,,,,,,,,,,,,,,,,,
AAAS,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
AACS,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AADAC,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
AAK1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
AAMP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
df_human2.shape

(9760, 184)

In [12]:
# create the hypergraph using from_dataframe with no additional arguments
H_b_filter_hand = hnx.Hypergraph.from_dataframe(df_human2)

In [13]:
H_b_filter_hand.shape

(8072, 174)

### Option 3: using the hnx.Hypergraph.from_dataframe() command

In [14]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
H_b_filter_shortcut = hnx.Hypergraph.from_dataframe(df, # the whole dataframe, b and p columns
                                                    columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [15]:
H_b_filter_shortcut.shape

(8072, 174)

## Centrality:
All three of the hypergraphs we created have genes as vertices and conditions as edges. In order to apply the centrality functions and get a ranking of genes we must work on the dual hypergraph, where genes are edges and conditions are vertices. We use the dual() function from HNX. We'll just use the H_b_filter_shortcut hypergraph from now on.

In [16]:
Hd = H_b_filter_shortcut.dual()

In [17]:
Hd.shape

(174, 8072)

There are two centrality commands, one for betweenness and one for closeness. Both take a hypergraph and an s value parameter and return dictionaries of centrality values for each gene. 

In [18]:
# running example here for large s value because it finishes relatively quickly. 
# Small s values take a long time on these large hypergraphs!
betcen30 = hnx.s_betweenness_centrality(Hd, s=30)
clocen30 = hnx.s_harmonic_closeness_centrality(Hd, s=30)

In [19]:
betcen30

{'ABL2': 0.00026976442221506405,
 'GBP1': 0.0013143505898887012,
 'USP18': 0.011367100469827524,
 'BNIP3L': 0.0,
 'CXCL2': 0.08884148530281176,
 'IRF7': 0.0023792123107164334,
 'PER1': 0.0006159906023165024,
 'CCL2': 9.267430116217106e-06,
 'FTMT': 0.0,
 'DHRS9': 0.0,
 'OVOL1': 0.0,
 'KIFC3': 0.001914656917601532,
 'GADD45A': 0.0,
 'CDK5R2': 0.0037167490125236554,
 'SLC6A4': 0.00013836175736294787,
 'EIF5': 0.0001183382489569103,
 'SYNPO2': 0.0,
 'PTX3': 0.0011361202543553701,
 'KLF10': 0.002304394890316857,
 'ARMC7': 0.0,
 'PDZD2': 0.0,
 'MXD1': 0.0,
 'BBC3': 0.0,
 'PLK2': 0.0,
 'ZFP36': 1.2070376092756027e-05,
 'RHOF': 1.973673957674588e-05,
 'CCRN4L': 0.0,
 'NR1D1': 0.0018574900848695865,
 'EPSTI1': 0.0021007451506290004,
 'LTA': 0.0,
 'DPYSL5': 0.0,
 'CLU': 0.0,
 'PNPLA5': 0.0008046869307237473,
 'MAFF': 0.0,
 'RSAD2': 0.01313674526904222,
 'ZZZ3': 0.0003681090457920651,
 'ATP1A3': 0.0017937749309605384,
 'IFIT2': 0.024928977125766607,
 'FOSB': 9.264802866067127e-05,
 'ATF3': 0.225

In [20]:
clocen30

{'CD274': 0.4492753623188405,
 'SGIP1': 0.0,
 'SLC45A2': 0.0,
 'SCN3A': 0.0,
 'IL15RA': 0.34210526315789475,
 'SYNPO2': 0.3155987795575895,
 'MAK': 0.3239893211289093,
 'RASD1': 0.34820747520976353,
 'VSIG2': 0.42601067887109073,
 'IFITM1': 0.4965675057208238,
 'COL9A2': 0.0,
 'LGALS9': 0.3424866514111366,
 'CD86': 0.32799389778794813,
 'KCNH2': 0.0,
 'KLRC1': 0.0,
 'CEL': 0.36842105263157904,
 'RASGRP4': 0.0,
 'RGS4': 0.32170099160945836,
 'ARHGEF15': 0.0,
 'SIDT1': 0.3905415713196034,
 'RASSF4': 0.33600305110602596,
 'STMN4': 0.0,
 'KRT83': 0.0,
 'OAS2': 0.5205949656750573,
 'RTN3': 0.0,
 'IFNA4': 0.0,
 'GSG1': 0.0,
 'FTMT': 0.339816933638444,
 'KALRN': 0.0,
 'MX2': 0.5148741418764302,
 'FBN2': 0.33142639206712443,
 'SH3BP1': 0.0,
 'TTLL6': 0.3592677345537758,
 'HSH2D': 0.40045766590389015,
 'ACTN2': 0.3630816170861938,
 'PSMD10': 0.0,
 'COX6B2': 0.0,
 'COL11A2': 0.3215102974828376,
 'RAC2': 0.0,
 'IL17RE': 0.0,
 'SPARC': 0.0,
 'RTP4': 0.4344012204424104,
 'RASD2': 0.0,
 'LTBP4': 0.0